# Segmenting and Clustering Neighborhoods in Toronto

In [48]:
#Imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
import folium 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Data collection and cleaning

In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
soup = BeautifulSoup(r.content)

In [8]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [9]:
df = pd.DataFrame(table_contents)

In [10]:
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')

In [11]:
finaldf = df.merge(geospatial_data, left_on='PostalCode', right_on='Postal Code', indicator=False)
finaldf = finaldf.drop(columns=['Postal Code'])
finaldf

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## 2. Map plotting

In [39]:
#Create initial map
map_toronto = folium.Map(location=[43.65, -79.38], zoom_start=10)

#Add markers to map
for lat, lng, borough, neighborhood in zip(finaldf['Latitude'], finaldf['Longitude'], finaldf['Borough'], finaldf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

#Final map
map_toronto

## 3. Data Collection from API Square

In [12]:
CLIENT_ID = 'S3SCFNO5VLZDOXGRZQ5OOLOPNGR1BYBMEAMHDR4QZWXM0MN0' # your Foursquare ID
CLIENT_SECRET = 'IEHTWO4PJKFUAGF52MLVQQDPCKXMMU3CAGIEZYRVANBL5IRA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    finaldf.loc[0,'Latitude'], 
    finaldf.loc[0,'Longitude'], 
    radius, 
    LIMIT)

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=finaldf['Neighborhood'], latitudes=finaldf['Latitude'], longitudes=finaldf['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [34]:
#Add one hot encoding variable and add the neighborhood name as the first column
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])

In [35]:
toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
#Instead of using the average as in the NY case, I am using the sum.
#This will be the Data Frame employed for the cluster analysis
toronto_grouped = toronto_onehot.groupby('Neighborhood').sum().reset_index()

In [53]:
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bayview Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,"Willowdale, Newtonbrook",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,Woodbine Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
#Printing the top 5 most frequent venue in each neighborhood.
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

       Gas Station   1.0
1   Fried Chicken Joint   1.0
2                Bakery   1.0
3       Thai Restaurant   1.0
4  Caribbean Restaurant   1.0


----Central Bay Street----
                venue  freq
0         Coffee Shop  11.0
1      Sandwich Place   4.0
2                Café   3.0
3  Italian Restaurant   3.0
4        Burger Joint   2.0


----Christie----
           venue  freq
0  Grocery Store   4.0
1           Café   3.0
2           Park   2.0
3     Baby Store   1.0
4    Coffee Shop   1.0


----Church and Wellesley----
                 venue  freq
0          Coffee Shop   6.0
1     Sushi Restaurant   5.0
2  Japanese Restaurant   5.0
3              Gay Bar   3.0
4           Restaurant   3.0


----Clairville, Humberwood, Woodbine Downs, West Humber, Kipling Heights, Rexdale, Elms, Tandridge, Old Rexdale----
                 venue  freq
0           Truck Stop   1.0
1        Garden Center   1.0
2                  Bar   1.0
3  Rental Car Location   1.0
4            Drugstore   1.0


--

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Clothing Store,Breakfast Spot,Lounge,Latin American Restaurant,Skating Rink
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pub,Coffee Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Restaurant,Middle Eastern Restaurant
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Accessories Store
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Comfort Food Restaurant,Hobby Shop


## 4. Cluster Analysis

In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

<ipython-input-41-7d20be48104f>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


array([2, 2, 2, 2, 0, 3, 2, 0, 2, 2])

In [60]:
kmeans.labels_

array([2, 2, 2, 2, 0, 3, 2, 0, 2, 2, 2, 3, 2, 3, 2, 2, 2, 1, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 4, 1, 2, 4, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       2, 3, 1, 2, 2, 2, 0, 2, 2, 2, 3, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 1,
       0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2])

### Clusters visualisation

Data frame required for the final visualization

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [63]:
toronto_merged = finaldf

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged[toronto_merged['Cluster Labels'].notna()]

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Fast Food Restaurant,Bus Stop,Food & Drink Shop,Park,Accessories Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,2.0,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,French Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3.0,Coffee Shop,Pub,Park,Bakery,Theater
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Clothing Store,Accessories Store,Event Space,Boutique,Furniture / Home Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Theater,Mexican Restaurant


In [65]:
# create map
map_clusters = folium.Map(location=[43.65, -79.38], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examination of the clusters

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Queen's Park,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Theater,Mexican Restaurant
23,East York,0.0,Coffee Shop,Sporting Goods Shop,Burger Joint,Sushi Restaurant,Bank
37,West Toronto,0.0,Bar,Men's Store,Vietnamese Restaurant,Restaurant,Coffee Shop
41,East Toronto,0.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store
43,West Toronto,0.0,Café,Bakery,Breakfast Spot,Coffee Shop,Grocery Store
54,East Toronto,0.0,Coffee Shop,Brewery,American Restaurant,Gastropub,Café
55,North York,0.0,Italian Restaurant,Coffee Shop,Sandwich Place,Comfort Food Restaurant,Hobby Shop
59,North York,0.0,Ramen Restaurant,Pizza Place,Coffee Shop,Café,Shopping Mall
74,Central Toronto,0.0,Sandwich Place,Café,Coffee Shop,History Museum,BBQ Joint
79,Central Toronto,0.0,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Sushi Restaurant


In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
30,Downtown Toronto,1.0,Coffee Shop,Café,Clothing Store,Thai Restaurant,Restaurant
36,Downtown Toronto,1.0,Coffee Shop,Aquarium,Hotel,Café,Brewery
42,Downtown Toronto,1.0,Coffee Shop,Hotel,Café,Restaurant,Salad Place
48,Downtown Toronto,1.0,Coffee Shop,Restaurant,Café,Hotel,Gym
97,Downtown Toronto,1.0,Coffee Shop,Café,Hotel,Gym,Japanese Restaurant


In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,2.0,Fast Food Restaurant,Bus Stop,Food & Drink Shop,Park,Accessories Store
1,North York,2.0,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,French Restaurant
3,North York,2.0,Clothing Store,Accessories Store,Event Space,Boutique,Furniture / Home Store
6,Scarborough,2.0,Fast Food Restaurant,Accessories Store,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark
7,North York,2.0,Japanese Restaurant,Caribbean Restaurant,Café,Dessert Shop,Gym
...,...,...,...,...,...,...,...
94,EtobicokeNorthwest,2.0,Rental Car Location,Truck Stop,Bar,Drugstore,Garden Center
98,Etobicoke,2.0,Pool,River,Accessories Store,Miscellaneous Shop,Moroccan Restaurant
100,East TorontoBusiness reply mail Processing Cen...,2.0,Yoga Studio,Restaurant,Spa,Pizza Place,Fast Food Restaurant
101,Etobicoke,2.0,Baseball Field,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,2.0,Fast Food Restaurant,Bus Stop,Food & Drink Shop,Park,Accessories Store
1,North York,2.0,Portuguese Restaurant,Hockey Arena,Intersection,Coffee Shop,French Restaurant
3,North York,2.0,Clothing Store,Accessories Store,Event Space,Boutique,Furniture / Home Store
6,Scarborough,2.0,Fast Food Restaurant,Accessories Store,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark
7,North York,2.0,Japanese Restaurant,Caribbean Restaurant,Café,Dessert Shop,Gym
...,...,...,...,...,...,...,...
94,EtobicokeNorthwest,2.0,Rental Car Location,Truck Stop,Bar,Drugstore,Garden Center
98,Etobicoke,2.0,Pool,River,Accessories Store,Miscellaneous Shop,Moroccan Restaurant
100,East TorontoBusiness reply mail Processing Cen...,2.0,Yoga Studio,Restaurant,Spa,Pizza Place,Fast Food Restaurant
101,Etobicoke,2.0,Baseball Field,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark


In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,Downtown Toronto,4.0,Coffee Shop,Clothing Store,Café,Japanese Restaurant,Cosmetics Shop
33,North York,4.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Cosmetics Shop,Food Court
